# 0. Чтение данных и базовое ознакомление

In [152]:
# импортируем необходимые библиотеки
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

In [153]:
# считываем данные
df = pd.read_csv('data/data.csv')

In [154]:
# выясним размерность датасета
print('размерность датасета: ',df.shape)
# проверим корректность загрузки и ознакомимся с полями
df.head()

размерность датасета:  (377185, 18)


,status,private pool,propertyType,street,baths,homeFacts,fireplace,city,schools,sqft,zipcode,beds,state,stories,mls-id,PrivatePool,MlsId,target
0,Active,NaN,Single Family Home,240 Heather Ln,3.5,"{'atAGlanceFacts': [{'factValue': '2019', 'fac...",Gas Logs,Southern Pines,"[{'rating': ['4', '4', '7', 'NR', '4', '7', 'N...",2900,28387,4,NC,NaN,NaN,NaN,611019,"$418,000"
1,for sale,NaN,single-family home,12911 E Heroy Ave,3 Baths,"{'atAGlanceFacts': [{'factValue': '2019', 'fac...",NaN,Spokane Valley,"[{'rating': ['4/10', 'None/10', '4/10'], 'data...","1,947 sqft",99216,3 Beds,WA,2.0,NaN,NaN,201916904,"$310,000"
2,for sale,NaN,single-family home,2005 Westridge Rd,2 Baths,"{'atAGlanceFacts': [{'factValue': '1961', 'fac...",yes,Los Angeles,"[{'rating': ['8/10', '4/10', '8/10'], 'data': ...","3,000 sqft",90049,3 Beds,CA,1.0,NaN,yes,FR19221027,"$2,895,000"
3,for sale,NaN,single-family home,4311 Livingston Ave,8 Baths,"{'atAGlanceFacts': [{'factValue': '2006', 'fac...",yes,Dallas,"[{'rating': ['9/10', '9/10', '10/10', '9/10'],...","6,457 sqft",75205,5 Beds,TX,3.0,NaN,NaN,14191809,"$2,395,000"
4,for sale,NaN,lot/land,1524 Kiscoe St,NaN,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",NaN,Palm Bay,"[{'rating': ['4/10', '5/10', '5/10'], 'data': ...",NaN,32908,NaN,FL,NaN,NaN,NaN,861745,"$5,000"


In [155]:
# оценим количество пропусков и типы данных
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 377185 entries, 0 to 377184
Data columns (total 18 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   status        337267 non-null  object
 1   private pool  4181 non-null    object
 2   propertyType  342452 non-null  object
 3   street        377183 non-null  object
 4   baths         270847 non-null  object
 5   homeFacts     377185 non-null  object
 6   fireplace     103115 non-null  object
 7   city          377151 non-null  object
 8   schools       377185 non-null  object
 9   sqft          336608 non-null  object
 10  zipcode       377185 non-null  object
 11  beds          285903 non-null  object
 12  state         377185 non-null  object
 13  stories       226470 non-null  object
 14  mls-id        24942 non-null   object
 15  PrivatePool   40311 non-null   object
 16  MlsId         310305 non-null  object
 17  target        374704 non-null  object
dtypes: object(18)
memory usa

In [156]:
# посмотрим на количество явных пропусков с более удобного ракурса
df.isnull().sum()

status           39918
private pool    373004
propertyType     34733
street               2
baths           106338
homeFacts            0
fireplace       274070
city                34
schools              0
sqft             40577
zipcode              0
beds             91282
state                0
stories         150715
mls-id          352243
PrivatePool     336874
MlsId            66880
target            2481
dtype: int64

Описание данных:

- 'status' — статус продажи;
- 'private pool' и 'PrivatePool' — наличие собственного бассейна;
- 'propertyType' — тип объекта недвижимости;
- 'street' — адрес объекта;
- 'baths' — количество ванных комнат;
- 'homeFacts' — сведения о строительстве объекта (содержит несколько типов сведений, влияющих на оценку объекта);
- 'fireplace' — наличие камина;
- 'city' — город;
- 'schools' — сведения о школах в районе;
- 'sqft' — площадь в футах;
- 'zipcode' — почтовый индекс;
- 'beds' — количество спален;
- 'state' — штат;
- 'stories' — количество этажей;
- 'mls-id' и 'MlsId' — идентификатор MLS (Multiple Listing Service, система мультилистинга);
- 'target' — цена объекта недвижимости (целевой признак, который необходимо спрогнозировать).

_____________

# 1. Предобработка данных

In [157]:
# для начала удалим записи с пустыми значениями целевой переменной, они явно не участвуют в этом мероприятии
df = df[~df['target'].isna()]
df.shape[0]

374704

In [158]:
# пройдемся по датасету и уберем общую проблему - лишние пробелы и переносы строк
df = df.replace({r'\s+$': '', r'^\s+': ''}, regex=True).replace(r'\n',  ' ', regex=True)

In [159]:
# проанализируем датасет на наличие полных дубликатов
df.duplicated().sum()

49

In [160]:
# удалим полные дубликаты исходного датасета
df = df.drop_duplicates()
df.shape[0]

374655

#### Последовательно проверим признаки

In [161]:
# ознакомимся с внесенными значениями по диагонали, чтобы выделить основные сложности с обработкой
columns = df.columns
for column in columns:
    print()
    print(column)
    print('количество вариантов : ', df[column].value_counts().shape[0])
    print()
    print(df[column].value_counts().head(50))
    if df[column].value_counts().shape[0] > 50:
        print(df[column].value_counts().tail(50))


status
количество вариантов :  156

for sale                             156054
Active                               105206
For sale                              43464
foreclosure                            5677
New construction                       5458
Pending                                4697
Pre-foreclosure                        2000
P                                      1488
Pre-foreclosure / auction              1281
Under Contract Show                    1183
/ auction                               799
Under Contract   Showing                793
Active Under Contract                   718
New                                     690
Under Contract                          690
Contingent                              581
Price Change                            563
Auction                                 493
A Active                                443
for rent                                398
Foreclosure                             343
Foreclosed                             

##### Первичный осмотр признаков

- status

156 вариантов - возможно просмотреть и обработать детально. 
Какие-то категории точно можно объединять, вроде “Coming soon”
При этом, объект может менять свои статусы, и какие-то могут влиять на цену (например, как предположение, за долги), а какие-то нет (когда проходит обычный жизненный цикл вроде такого: “для продажи - показы - договор без обязательств”)

- propertyType

Один из самых сложных признаков в выборке. 
Разное написание, разные сокращения, и перечисление признаков через запятые и слэш, а где-то и просто через пробел. Очень неудобно.
Даже при разложении каждой записи этого поля на отдельные составляющие вложенного списка пока конечное количество вариантов непонятно.

- street

нет цели проверить каждый адрес, но топ повторяющихся вариантов можно и нужно просмотреть как минимум на предмет разных написаний “адреса нет”
Навскидку по крайней мере похоже, что в поле и правда вносят адрес.

- baths

Вариантов 226. Многовато, но можно обработать массовые нюансы - убрать типовые наборы слов в начале или конце записи, после чего повторно посмотреть схлопнувшиеся варианты и доработать еще раз.
Уже видны записи с 76 и 241 ванной. Пока тяжело осознать, что это может быть, если  не выброс. Также присутствуют записи через слэш. Надо смотреть подробнее.

- homefacts

Пока только понятно, что надо разложить на отдельные поля этот вложенный список. После этого уже отдельно смотреть что оттуда вытянется.

- fireplace

Разное написание, и где-то цифры, где-то слова. 1653 варианта - много для полноценной ручной обработки с индивидуальным просмотром.
При этом, наблюдаются характерные записи с указанием, газ это, электрика или на дровах/брикетах.
Нужно уменьшать количество вариантов.

- city

2019 нас.пункта. В первом приближении похоже, что внесены города.
Есть вероятность разного написания, но это можно выяснить только при более детальном просмотре.

- schools

списки разной длины, с разным количеством школ и соответствующих им характеристик. Здесь вариант только приводить к какой-то средней оценке. 

- sqft

Площадь объекта - по ощущениям основной показатель, его следует обработать максимально аккуратно.
Видно, что присутствует различное написание единиц измерения, а где-то ЕИ совсем отсутствует. Где-то есть разделитель разрядов, где-то его нет. Есть отдельные записи и с текстовым описанием. 
И конечно, критичны записи с отсутствием информации. Учитывая неаккуратность датасета, эта информация может присутствовать в других полях, необходимо посмотреть подробнее.

- zipcode 

Порадовало, что в верхней части рейтинга популярности значений отсутствует “нет кода” или нулевое значение. Этот факт дает надежду, что сможем определять нахождение объекта более подробно, нежели штат или город. 
Но есть некие значения через дефис. Первая составная часть похожа на основную массу индексов, со второй (после дефиса) пока непонятно.

- beds

1147 вариантов. Разное написание/обозначение слова “кровать”. Но что удивительно, в этом поле кое-где внесены значения площади. Возможно, в форме для заполнения какие-то поля были рядом и неочевидно подписаны, за счет чего сюда вносилась информация о площади (то ли жилой, то ли участка). Надо смотреть.

- state

Выглядит аккуратно, но смущают единичные значения - неужели один объект во всем штате? Надо проверить.

- stories

Знаки плюсов, где-то значения с точками, где-то без точек, где-то словами вроде “One” и т.п. Чистить и смотреть результат.

- mls-id и MlsId

Много повторяющихся значений, хотя по описанию признака казалось, что это уникальный идентификатор. Похоже, что нет, надо посмотреть поближе.

- target

Собственно целевой признак. Есть со значком $, есть без него. Есть плюсы, которые даже если что-то обозначали, мы проигнорируем и удалим.
И бросается в глаза в топ-30 значение $1000 в количестве почти тысяча записей. Тоже надо посмотреть.


Приступим к обработке от простого к сложному

____________

##### Частный бассейн - 'private pool' и 'PrivatePool'

In [162]:
# проверим пару признаков наличия частного бассейна
# для начала проверим варианты внесения информации
display(df['private pool'].value_counts())
display(df['PrivatePool'].value_counts())

Yes    4151
Name: private pool, dtype: int64

yes    28686
Yes    11434
Name: PrivatePool, dtype: int64

In [163]:
# предположим, что это признаки, созданные в базе данных в разное время
# т.е., информация о наличии бассейна есть либо в одном, либо в другом
# в таком случае, не должно быть записей, в которых одновременно указано наличие в обоих полях
# проверим простым способом - переведем наличие в "1" и сложим оба столбца
# если запись и там, и там, получим в этих записях двойки

# для начала переведем записи в единицы и проверим сохранение количеств значений
df['PrivatePool'] = df['PrivatePool'].apply(lambda x: 1 if x in ['yes', 'Yes'] else 0)
display(df['PrivatePool'].value_counts())
df['private pool'] = df['private pool'].apply(lambda x: 1 if x in ['yes', 'Yes'] else 0)
display(df['private pool'].value_counts())

0    334535
1     40120
Name: PrivatePool, dtype: int64

0    370504
1      4151
Name: private pool, dtype: int64

In [164]:
# поскольку перекодирование столбцов прошло успешно, теперь делаем аггрегирующее поле
# проверка вариантов значений покажет, подтвердилось ли вышеописанное предположение
df['PoolPrivate'] = df['PrivatePool'] + df['private pool']
df['PoolPrivate'].value_counts()

0    330384
1     44271
Name: PoolPrivate, dtype: int64

In [165]:
# схема сработала, изначальные признаки можно удалить
df = df.drop(columns=['private pool', 'PrivatePool'], axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 374655 entries, 0 to 377184
Data columns (total 17 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   status        335399 non-null  object
 1   propertyType  340101 non-null  object
 2   street        374653 non-null  object
 3   baths         269308 non-null  object
 4   homeFacts     374655 non-null  object
 5   fireplace     102519 non-null  object
 6   city          374621 non-null  object
 7   schools       374655 non-null  object
 8   sqft          334560 non-null  object
 9   zipcode       374655 non-null  object
 10  beds          283726 non-null  object
 11  state         374655 non-null  object
 12  stories       224902 non-null  object
 13  mls-id        24937 non-null   object
 14  MlsId         310187 non-null  object
 15  target        374655 non-null  object
 16  PoolPrivate   374655 non-null  int64 
dtypes: int64(1), object(16)
memory usage: 51.5+ MB


__________

##### Идентификаторы MLS - 'mls-id' и 'MlsId'

In [166]:
# проверим пару признаков идентификаторов MLS
# предположение такое - признаки заполнялись в разные периоды времени (как бассейны)
# если это так, значение есть либо в одном поле, либо в другом, проверим
print('Количество записей с одновременно заполненными полями "MlsId" и "mls-id" : ',
      df[~df['MlsId'].isna() & ~df['mls-id'].isna()].shape[0])

Количество записей с одновременно заполненными полями "MlsId" и "mls-id" :  0


In [167]:
# нет ни одной записи, в которой одновременно были бы непустые значения в обоих полях
# теперь проверим, нет ли одинаковых идентификаторов в обоих полях, и если есть, то идентичные ли записи им соответствуют
# это будет обозначать, что в какой-то момент одно поле для внесения отключили, а второе активировали
# для начала переведем в строчные буквы
df['MlsId'] = df['MlsId'].str.lower()
df['mls-id'] = df['mls-id'].str.lower()
MlsIdList = list(df['MlsId'].unique())
print('уникальных значений MlsId : ',len(MlsIdList))
mls_id_List = list(df['mls-id'].unique())
print('уникальных значений mls-id : ',len(mls_id_List))
print ('при пересечении только nan длина множества должна быть : ', (len(MlsIdList)+len(mls_id_List)-1))
Mls = set(MlsIdList + mls_id_List)
print('уникальных значений кумулятивно по обоим столбцам : ',len(Mls))
if len(Mls) < (len(MlsIdList)+len(mls_id_List)-1):
    print('значения столбцов пересекаются')
else:
    print('значения столбцов не пересекаются')

уникальных значений MlsId :  232861
уникальных значений mls-id :  24902
при пересечении только nan длина множества должна быть :  257762
уникальных значений кумулятивно по обоим столбцам :  248925
значения столбцов пересекаются


In [168]:
# выделим несколько ID MLS, которые присутствуют в обоих полях, и проверим, одинаковые ли объекты им соответствуют
q = list(set(MlsIdList) & set(mls_id_List))[1:]
df[df['mls-id'].isin(q[:10]) | df['MlsId'].isin(q[:10])].sort_values(by='street')

,status,propertyType,street,baths,homeFacts,fireplace,city,schools,sqft,zipcode,beds,state,stories,mls-id,MlsId,target,PoolPrivate
374969,For sale,Single Family,(undisclosed Address),Bathrooms: 4,"{'atAGlanceFacts': [{'factValue': '2002', 'fac...",NaN,Las Vegas,"[{'rating': ['7/10', '7/10', '4/10'], 'data': ...","Total interior livable area: 3,314 sqft",89139,5 bd,NV,2.0,2148025,NaN,"$458,000",1
244924,For sale,Single Family,1500 N Terrell St,Bathrooms: 2,"{'atAGlanceFacts': [{'factValue': '2010', 'fac...",Yes,Midland,"[{'rating': ['4/10', '3/10', '3/10'], 'data': ...","Total interior livable area: 1,851 sqft",79701,3 bd,TX,1.0,50026716,NaN,"$320,000",0
220803,for sale,single-family home,1500 N Terrell St,2 Baths,"{'atAGlanceFacts': [{'factValue': '2010', 'fac...",yes,Midland,"[{'rating': ['3/10', '3/10', '4/10', '4/10'], ...","1,851 sqft",79701,3 Beds,TX,1.0,NaN,50026716,"$320,000",0
262020,For sale,Single Family,2182 Del Carmel Way,Bathrooms: 3,"{'atAGlanceFacts': [{'factValue': '2008', 'fac...",NaN,Tallahassee,"[{'rating': ['3/10', '2/10', '4/10'], 'data': ...","Total interior livable area: 1,228 sqft",32303,2 bd,FL,2.0,312877,NaN,"$95,000",1
158760,for sale,single-family home,2182 Del Carmel Way,3 Baths,"{'atAGlanceFacts': [{'factValue': '2008', 'fac...",NaN,Tallahassee,"[{'rating': ['4/10', '2/10', '3/10'], 'data': ...","1,228 sqft",32303,2 Beds,FL,2.0,NaN,312877,"$95,000",1
371470,for sale,condo,2182 Del Carmel Way #606,3 Baths,"{'atAGlanceFacts': [{'factValue': '2008', 'fac...",NaN,Tallahassee,"[{'rating': ['4/10', '2/10', '3/10'], 'data': ...","1,228 sqft",32303,2 Beds,FL,NaN,NaN,312877,"$95,000",1
248805,Active,NaN,3208 E Powhatan Ave,NaN,"{'atAGlanceFacts': [{'factValue': '1952', 'fac...",NaN,TAMPA,"[{'rating': ['1', '2', '3'], 'data': {'Distanc...",890,33610,NaN,FL,One,NaN,t3206785,"154,900",0
84312,for sale,single-family home,3208 E Powhatan Ave,NaN,"{'atAGlanceFacts': [{'factValue': '1952', 'fac...",NaN,Tampa,"[{'rating': ['2/10', '4/10', '1/10'], 'data': ...",890 sqft,33610,3 Beds,FL,1.0,NaN,t3206785,"$154,900",0
99341,For sale,Single Family,3208 E Powhatan Ave,Bathrooms: 1,"{'atAGlanceFacts': [{'factValue': '1952', 'fac...",NaN,Tampa,"[{'rating': ['1/10', '2/10', '4/10'], 'data': ...",Total interior livable area: 890 sqft,33610,3 bd,FL,1.0,t3206785,NaN,"$158,900",0
254221,For sale,Single Family,5512 Dartmouth Ave,2 ba,"{'atAGlanceFacts': [{'factValue': '1941', 'fac...",NaN,River Oaks,"[{'rating': ['5/10', '4/10', '4/10'], 'data': ...","1,404 sqft",76114,3 bd,TX,NaN,14194640,NaN,"$192,900",0


In [169]:
# на примере выборочной проверки делаем вывод, что идентификаторы в обоих полях относятся к одним и тем же объектам
# объединим идентификатор MLS в один столбец и удалим два исходных
df['MLS'] = df['MlsId'].fillna('') + df['mls-id'].fillna('')
df = df.drop(columns=['MlsId', 'mls-id'], axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 374655 entries, 0 to 377184
Data columns (total 16 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   status        335399 non-null  object
 1   propertyType  340101 non-null  object
 2   street        374653 non-null  object
 3   baths         269308 non-null  object
 4   homeFacts     374655 non-null  object
 5   fireplace     102519 non-null  object
 6   city          374621 non-null  object
 7   schools       374655 non-null  object
 8   sqft          334560 non-null  object
 9   zipcode       374655 non-null  object
 10  beds          283726 non-null  object
 11  state         374655 non-null  object
 12  stories       224902 non-null  object
 13  target        374655 non-null  object
 14  PoolPrivate   374655 non-null  int64 
 15  MLS           374655 non-null  object
dtypes: int64(1), object(15)
memory usage: 48.6+ MB


In [170]:
# посмотрим неоднократно встречающиеся варианты для проверки разного написания отсутствия номера
df['MLS'].value_counts().head(50)

                             39531
no mls                          42
no mls #                        16
a, houston, tx 77008            13
no                              12
12a, orlando, fl 32833          11
b, houston, tx 77008             9
1, south boston, ma 02127        9
b, houston, tx 77007             8
2, washington, dc 20002          8
11a, orlando, fl 32833           8
1, washington, dc 20002          7
2, washington, dc 20010          7
1, washington, dc 20010          6
1, washington, dc 20001          6
a, austin, tx 78721              6
2, washington, dc 20001          6
a, austin, tx 78704              6
2101941                          6
1412350                          6
2088662                          6
3a, orlando, fl 32833            6
1a, orlando, fl 32833            6
2, washington, dc 20009          6
nomlsid                          6
2, boston, ma 02129              6
0, doral, fl 33178               6
a, houston, tx 77018             5
14181176            

In [171]:
# заменим обозначение отсутствия номера MLS на однотипное пустое значение
no_mls = ['no mls', 'no mls #', 'no', 'nomlsid']
df['MLS'] = df['MLS'].apply(lambda x: np.nan if (x in no_mls) else x)
df['MLS'].value_counts().head(50)

                             39531
a, houston, tx 77008            13
12a, orlando, fl 32833          11
1, south boston, ma 02127        9
b, houston, tx 77008             9
2, washington, dc 20002          8
11a, orlando, fl 32833           8
b, houston, tx 77007             8
2, washington, dc 20010          7
1, washington, dc 20002          7
2, boston, ma 02129              6
1, washington, dc 20010          6
2, washington, dc 20001          6
0, doral, fl 33178               6
a, austin, tx 78704              6
1412350                          6
1, washington, dc 20001          6
3a, orlando, fl 32833            6
a, austin, tx 78721              6
2, washington, dc 20009          6
2088662                          6
1a, orlando, fl 32833            6
2101941                          6
b, houston, tx 77057             5
1026004                          5
2281272                          5
1019437                          5
14168541                         5
1367153             

In [172]:
# оценим количество пропусков
print(df['MLS'].isna().sum())
print(round((df['MLS'].isna().sum()/df.shape[0]*100),0),'%')

76
0.0 %


____________

##### Target - Целевой признак

In [173]:
# оценим пропуски и посмотрим варианты значений
print('пропусков : ',df['target'].isna().sum())
print('пропусков в %: ',round((df['target'].isna().sum()/df.shape[0]*100),0),'%')
print('уникальных значений : ',df['target'].nunique())
df['target'].value_counts()

пропусков :  0
пропусков в %:  0.0 %
уникальных значений :  43939


$225,000     1462
$275,000     1355
$250,000     1312
$350,000     1296
$299,900     1276
             ... 
274,359         1
$273,490+       1
$645,000+       1
$28,272         1
$171,306        1
Name: target, Length: 43939, dtype: int64

In [174]:
# проверим, нет ли помимо явных символов еще и букв в каких-то значениях
df[df['target'].str.contains('[a-zA-Z:]')]

,status,propertyType,street,baths,homeFacts,fireplace,city,schools,sqft,zipcode,beds,state,stories,target,PoolPrivate,MLS
547,for rent,single-family home,4323 N Central Park Ave,3.5 Baths,"{'atAGlanceFacts': [{'factValue': '1913', 'fac...",yes,Chicago,"[{'rating': ['1/10', '4/10', '2/10', 'None/10'...","3,300 sqft",60618,4 Beds,IL,NaN,"$5,500/mo",0,10588057
609,for rent,multi-family,220 Boylston St #1412,2 Baths,"{'atAGlanceFacts': [{'factValue': '1985', 'fac...",yes,Boston,"[{'rating': [], 'data': {'Distance': [], 'Grad...","1,673 sqft",2116,2 Beds,MA,NaN,"$10,500/mo",0,72580936
2075,for rent,single-family home,2830 NE 56th Ct,4 Baths,"{'atAGlanceFacts': [{'factValue': '1965', 'fac...",NaN,Fort Lauderdale,"[{'rating': ['6/10', '2/10', '4/10'], 'data': ...","2,400 sqft",33308,4 Beds,FL,NaN,"$6,390/mo",1,a10521855
3025,for rent,multi-family,411 Kline Aly,2.5 Baths,"{'atAGlanceFacts': [{'factValue': '2014', 'fac...",NaN,Clarksville,"[{'rating': ['8/10', '9/10', '7/10'], 'data': ...","1,280 sqft",37040,2 Beds,TN,NaN,"$1,200/mo",0,2102821
3645,for rent,multi-family,240 E Illinois St #2011,2 Baths,"{'atAGlanceFacts': [{'factValue': '2003', 'fac...",NaN,Chicago,"[{'rating': ['4/10', '7/10'], 'data': {'Distan...","1,473 sqft",60611,2 Beds,IL,NaN,"$3,600/mo",1,10590275
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
371791,for rent,multi-family,9436 Turrentine Dr,1.5 Baths,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",NaN,El Paso,"[{'rating': ['4/10', '8/10', '6/10'], 'data': ...","1,050 sqft",79925,2 Beds,TX,NaN,$890/mo,0,820163
372459,for rent,townhouse,34 Jonquil Pl,2.5 Baths,"{'atAGlanceFacts': [{'factValue': '2014', 'fac...",NaN,The Woodlands,"[{'rating': ['5/10', '8/10', '7/10', '8/10'], ...","2,601 sqft",77375,3 Beds,TX,NaN,"$2,500/mo",0,62158637
374288,for rent,single-family home,8864 Devonshire Dr,2 Baths,"{'atAGlanceFacts': [{'factValue': '2016', 'fac...",yes,Fort Worth,"[{'rating': ['6/10', '5/10', '5/10'], 'data': ...","2,000 sqft",76131,4 Beds,TX,NaN,"$2,000/mo",0,
375550,for rent,townhouse,2217 W Seybert St,NaN,"{'atAGlanceFacts': [{'factValue': '1920', 'fac...",NaN,Philadelphia,"[{'rating': ['1/10', '3/10'], 'data': {'Distan...",720 sqft,19121,2 Beds,PA,NaN,"$1,500/mo",0,paph857944


In [175]:
# обнаружились записи со стоимостью аренды в месяц, посмотрим в целом имеющие отношение к аренде записи
df[df['status'].str.contains('rent', na=False)]['status'].value_counts()

for rent              398
Apartment for rent      7
Condo for rent          7
Name: status, dtype: int64

In [176]:
# поскольку в целом модель должна предказывать стоимость продажи объекта
# плюс поскольку даже при желании на 400 записях достойный прогноз не построишь
# и эти записи составляют 0,1% от общего количества
# удаляем все записи, связанные с арендой
df = df[~df['status'].str.contains('rent', na=False)]
df.shape[0]

374243

In [177]:
# теперь заменим все остальные обнаруженные знаки, которые мешают перевести суммы в числовой формат
df['target'] = df['target'].apply(lambda x: int(x.replace('$','').
                                replace('+','').
                                replace(',','')))
# и собственно сменим тип данных в этом признаке
df['target'] = df['target'].astype(int)

________________

##### State - штат

In [178]:
# оценим пропуски и посмотрим варианты значений
print('пропусков : ',df['state'].isna().sum())
print('пропусков в %: ',round((df['state'].isna().sum()/df.shape[0]*100),0),'%')
print('уникальных значений : ',df['state'].nunique())
df['state'].value_counts()

пропусков :  0
пропусков в %:  0.0 %
уникальных значений :  39


FL    114548
TX     83263
NY     24324
CA     23169
NC     21760
TN     18217
WA     13721
OH     12422
IL      8821
NV      8401
GA      6628
CO      6371
PA      5493
MI      5119
DC      4580
AZ      3347
IN      3279
OR      2774
MA      1493
UT      1319
MD      1086
VT       864
MO       832
VA       800
WI       452
NJ       436
ME       258
IA       242
KY        90
OK        49
MS        40
SC        28
MT         7
DE         5
Fl         1
BA         1
AL         1
OT         1
OS         1
Name: state, dtype: int64

In [179]:
# в предположении, что индекс должен относиться только к одному штату, проверим соответствие индексов и штатов
# выделим те индексы, к которым сопоставлены более 1 штата
states = df.groupby(['zipcode', 'state'])['status'].agg('count').reset_index()[['zipcode', 'state']]
states1 = states.groupby('zipcode').agg('count').reset_index()
states2 = states1[states1['state'] > 1]
states2

,zipcode,state
0,--,2
1,0,3
197,11210,2
538,20003,2
1482,33179,2
1512,33321,2
2997,77380,2
3050,77710,2
3155,78501,2
3956,92703,2


In [180]:
# зафиксируем перечень индексов, к которым сопоставлены несколько штатов
states3 = list(states2['zipcode'][2:])
states3

['11210', '20003', '33179', '33321', '77380', '77710', '78501', '92703']

In [181]:
# теперь по выбранному перечню проверим соотношение количества записей, относящихся к тому или иному штату
df[df['zipcode'].isin(states3)].pivot_table('target', 'zipcode','state', 'count','')

state,BA,CA,DC,DE,FL,Fl,MA,NY,OT,TN,TX
zipcode,,,,,,,,,,,
11210,,,,,,,,194.0,1.0,,
20003,,,188.0,,,,1.0,,,,
33179,1.0,,,,682.0,,,,,,
33321,,,,,561.0,1.0,,,,,
77380,,,,,,,,,,1.0,124.0
77710,,,,3.0,,,,,,,5.0
78501,,,,,2.0,,,,,,165.0
92703,,84.0,,,,,,,,2.0,


In [182]:
# несоответствия очевидны (кроме 77710, интернет отнес к TX Техас), делаем замены
zipcode_replace = {'11210': 'NY',
                   '20003': 'DC',
                   '33179': 'FL',
                   '33321': 'FL',
                   '77380': 'TX',
                   '77710': 'TX',
                   '78501': 'TX',
                   '92703': 'CA'}

In [183]:
# напишем и применим функцию для замены
def zip_change(zip, state):
    if zip in zipcode_replace.keys():
        return zipcode_replace.get(zip)
    else:
        return state
    
df['state'] = df.apply(lambda row: zip_change(row['zipcode'], row['state']), axis=1)

In [184]:
# проверим себя, повторно проведем те же операции по проверке соответствия одному индексу одного штата
states = df.groupby(['zipcode', 'state'])['status'].agg('count').reset_index()[['zipcode', 'state']]
states1 = states.groupby('zipcode').agg('count').reset_index()
states2 = states1[states1['state'] > 1]
states2

,zipcode,state
0,--,2
1,0,3


In [185]:
# дополнительно отнесем в группу Other штаты, в которых количество записей менее 100
other = ['KY', 'OK', 'MS', 'SC', 'MT', 'DE', 'Fl', 'BA', 'AL', 'OT', 'OS']
df['state'] = df['state'].apply(lambda x: 'Other' if x in other else x)
df['state'].value_counts()

FL       114548
TX        83269
NY        24325
CA        23171
NC        21760
TN        18214
WA        13721
OH        12422
IL         8821
NV         8401
GA         6628
CO         6371
PA         5493
MI         5119
DC         4581
AZ         3347
IN         3279
OR         2774
MA         1492
UT         1319
MD         1086
VT          864
MO          832
VA          800
WI          452
NJ          436
ME          258
IA          242
Other       218
Name: state, dtype: int64

_____________

##### City - город

In [186]:
# оценим пропуски и посмотрим варианты значений
print('пропусков : ',df['city'].isna().sum())
print('пропусков в %: ',round((df['city'].isna().sum()/df.shape[0]*100),0),'%')
print('уникальных значений : ',df['city'].nunique())
df['city'].value_counts()

пропусков :  34
пропусков в %:  0.0 %
уникальных значений :  2019


Houston            24387
San Antonio        15496
Miami              15381
Jacksonville        9890
Dallas              8749
                   ...  
Los Altos Hills        1
Lake worth             1
Lisle                  1
Arrington              1
Blue Springs           1
Name: city, Length: 2019, dtype: int64

In [187]:
# в предположении, что индекс должен относиться только к одному городу, проверим соответствие индексов и городов
# выделим те индексы, к которым сопоставлены более 1 значения города
df['city'] = df['city'].str.lower()
cities = df.groupby(['zipcode', 'city'])['target'].agg('count').reset_index()[['zipcode', 'city']]
cities1 = cities.groupby('zipcode').agg('count').reset_index()
cities2 = cities1[cities1['city'] > 1][2:].reset_index().drop('index', axis=1)
cities2

,zipcode,city
0,02119,2
1,02122,2
2,02124,3
3,02125,2
4,02127,2
...,...,...
991,98908,2
992,99206,2
993,99208,2
994,99212,3


Достаточно много разночтений для индивидуального просмотра. Массово идей нет,  
т.к. к разным индексам привязаны разные лидеры написания одного и того же города.  
Пока оставляем как есть.

______________

##### street - адрес объекта

In [188]:
# оценим пропуски и посмотрим варианты значений
print('пропусков : ',df['street'].isna().sum())
print('пропусков в %: ',round((df['street'].isna().sum()/df.shape[0]*100),0),'%')
print('уникальных значений : ',df['street'].nunique())
df['street'].value_counts()

пропусков :  2
пропусков в %:  0.0 %
уникальных значений :  298648


Address Not Disclosed      672
Undisclosed Address        516
(undisclosed Address)      391
Address Not Available      175
Unknown Address             72
                          ... 
1346 Midland Ave APT 3J      1
302 Tempranillo Way          1
8288 Mount Nido Dr           1
25230 Cadiz Dr               1
7810 Pereida St              1
Name: street, Length: 298648, dtype: int64

In [189]:
# заменим разные вариации отсутствия адреса на единое значение
no_street = ['Address Not Disclosed', 
             'Undisclosed Address', 
             '(undisclosed Address)', 
             'Address Not Available', 
             'Unknown Address']
df['street'] = df['street'].apply(lambda x: np.nan if x in no_street else x)
df['street'].value_counts()

2103 E State Hwy 21        57
11305 Gulf Fwy             54
17030 Youngblood Rd.       38
NE 58th Cir                34
9470 Lancaster Rd. SW      32
                           ..
1346 Midland Ave APT 3J     1
302 Tempranillo Way         1
8288 Mount Nido Dr          1
25230 Cadiz Dr              1
7810 Pereida St             1
Name: street, Length: 298643, dtype: int64

По хорошему надо извлечь координаты с помощью библиотеки Geopy, но в данном случае технические ресурсы не позволяют это сделать.

___________

##### baths - количество ванных комнат

In [190]:
# оценим пропуски и посмотрим варианты значений
print('пропусков : ',df['baths'].isna().sum())
print('пропусков в %: ',round((df['baths'].isna().sum()/df.shape[0]*100),0),'%')
print('уникальных значений : ',df['baths'].nunique())
df['baths'].value_counts()

пропусков :  105250
пропусков в %:  28.0 %
уникальных значений :  225


2 Baths          51945
3 Baths          35353
2                20428
2.0              16354
4 Baths          14711
                 ...  
32                   1
5.25 Baths           1
41.0                 1
Bathrooms: 21        1
44.0                 1
Name: baths, Length: 225, dtype: int64

In [191]:
# проблему составляют буквы, пробелы и двоеточия, а также запятые в качестве разделителя. Произведем замену
df['baths'] = df['baths'].str.replace('Sq. Ft.','')
df['baths'] = df['baths'].str.replace('[a-zA-Z]','', regex=True)
df['baths'] = df['baths'].str.replace(' ','')
df['baths'] = df['baths'].str.replace(':','')
df['baths'] = df['baths'].str.replace('+','')
df['baths'] = df['baths'].str.replace('~','')
df['baths'] = df['baths'].str.replace(',','.')
df['baths'] = df['baths'].str.replace('2-1/2-1/1-1/1-1','4')
df['baths'] = df['baths'].str.replace('1/1-0/1-0/1-0','1')
df['baths'] = df['baths'].str.replace('1-0/1-0/1','1')
df['baths'] = df['baths'].str.replace('1/1/1/1','4')
df['baths'] = df['baths'].str.replace('3-1/2-2','3')
df['baths'] = df['baths'].str.replace('0/0','0')
df['baths'] = df['baths'].str.replace('116/116/116','116')
df['baths'] = df['baths'].str.replace('--','')
df['baths'] = df['baths'].str.replace('—','')
df['baths'] = df['baths'].apply(lambda x: 0 if x=='' else x)
df['baths'] = df['baths'].astype(float)
df['baths'].value_counts()

/var/folders/0v/d10yh6tj1mn387jzy55lmxv80000gn/T/ipykernel_1061/3676306326.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['baths'] = df['baths'].str.replace('Sq. Ft.','')
/var/folders/0v/d10yh6tj1mn387jzy55lmxv80000gn/T/ipykernel_1061/3676306326.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['baths'] = df['baths'].str.replace('+','')


2.0      102613
3.0       66381
4.0       26229
1.0       17520
2.5       13687
          ...  
14.5          1
5.2           1
116.0         1
35.0          1
68.0          1
Name: baths, Length: 81, dtype: int64

In [192]:
# Для отсутствующих значений укажем нулевые значения
df['baths'] = df['baths'].fillna(0)

_______________

##### Stories - количество этажей

In [193]:
# оценим пропуски и посмотрим варианты значений
print('пропусков : ',df['stories'].isna().sum())
print('пропусков в %: ',round((df['stories'].isna().sum()/df.shape[0]*100),0),'%')
print('уникальных значений : ',df['stories'].nunique())
df['stories'].value_counts()

пропусков :  149343
пропусков в %:  40.0 %
уникальных значений :  348


1.0                                  66805
2.0                                  55002
1                                    22793
2                                    17976
3.0                                  11230
                                     ...  
Manufactured Home, Non-Site Built        1
Bedroom - Split Plan                     1
78                                       1
None                                     1
65.0                                     1
Name: stories, Length: 348, dtype: int64

In [194]:
df[df['stories'].isna()]['propertyType'].value_counts()

single-family home                                             18973
lot/land                                                       18566
condo                                                          10950
Land                                                           10308
Single Family Home                                              9370
                                                               ...  
Single Detached, Contemporary/Modern, Southwestern                 1
Traditional, Texas Hill Country                                    1
Historical/Conservation District, Single Detached, Colonial        1
Bungalow, Transitional                                             1
Bilevel, Converted Dwelling, Loft with Bedrooms, Condo/Unit        1
Name: propertyType, Length: 1007, dtype: int64

Данный признак достойно не обработать, т.к. типы объектов недвижимости, для которых отсутствует  
информация об этажности, имеют большую вариативность количества этажей.  
То есть, на основании типа объекта сделать выводы об этажности не представляется возможным.  
Придется удалить признак.

In [195]:
df = df.drop('stories', axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 374243 entries, 0 to 377184
Data columns (total 15 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   status        334987 non-null  object 
 1   propertyType  339689 non-null  object 
 2   street        372415 non-null  object 
 3   baths         374243 non-null  float64
 4   homeFacts     374243 non-null  object 
 5   fireplace     102425 non-null  object 
 6   city          374209 non-null  object 
 7   schools       374243 non-null  object 
 8   sqft          334168 non-null  object 
 9   zipcode       374243 non-null  object 
 10  beds          283361 non-null  object 
 11  state         374243 non-null  object 
 12  target        374243 non-null  int64  
 13  PoolPrivate   374243 non-null  int64  
 14  MLS           374167 non-null  object 
dtypes: float64(1), int64(2), object(12)
memory usage: 45.7+ MB


___________

##### zipcode - почтовый индекс

In [196]:
# косвенно встреча с индексами была ранее при обработке признаков city и state
# оценим пропуски и посмотрим варианты значений
print('пропусков : ',df['zipcode'].isna().sum())
print('пропусков в %: ',round((df['zipcode'].isna().sum()/df.shape[0]*100),0),'%')
print('уникальных значений : ',df['zipcode'].nunique())

пропусков :  0
пропусков в %:  0.0 %
уникальных значений :  4541


In [197]:
# посмотрим некорректные индексы в шапке списка
df['zipcode'].sort_values().head(10)

235207       --
231282       --
30261         0
83522         0
305572        0
308229    00000
10837     02108
169423    02108
123565    02108
293976    02108
Name: zipcode, dtype: object

In [198]:
# удалим записи со спорными индексами
df = df[~(df['zipcode']=='--') & ~(df['zipcode']=='0') & ~(df['zipcode']=='00000')]
df.shape[0]

374237

In [199]:
# создадим вспомогательный столбец с количеством знаков в индексе, проверим по длине. По идее, везде должно быть 5
df['zip_len'] = df['zipcode'].apply(lambda x: len(x))
df.head()

,status,propertyType,street,baths,homeFacts,fireplace,city,schools,sqft,zipcode,beds,state,target,PoolPrivate,MLS,zip_len
0,Active,Single Family Home,240 Heather Ln,3.5,"{'atAGlanceFacts': [{'factValue': '2019', 'fac...",Gas Logs,southern pines,"[{'rating': ['4', '4', '7', 'NR', '4', '7', 'N...",2900,28387,4,NC,418000,0,611019,5
1,for sale,single-family home,12911 E Heroy Ave,3.0,"{'atAGlanceFacts': [{'factValue': '2019', 'fac...",NaN,spokane valley,"[{'rating': ['4/10', 'None/10', '4/10'], 'data...","1,947 sqft",99216,3 Beds,WA,310000,0,201916904,5
2,for sale,single-family home,2005 Westridge Rd,2.0,"{'atAGlanceFacts': [{'factValue': '1961', 'fac...",yes,los angeles,"[{'rating': ['8/10', '4/10', '8/10'], 'data': ...","3,000 sqft",90049,3 Beds,CA,2895000,1,fr19221027,5
3,for sale,single-family home,4311 Livingston Ave,8.0,"{'atAGlanceFacts': [{'factValue': '2006', 'fac...",yes,dallas,"[{'rating': ['9/10', '9/10', '10/10', '9/10'],...","6,457 sqft",75205,5 Beds,TX,2395000,0,14191809,5
4,for sale,lot/land,1524 Kiscoe St,0.0,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",NaN,palm bay,"[{'rating': ['4/10', '5/10', '5/10'], 'data': ...",NaN,32908,NaN,FL,5000,0,861745,5


In [200]:
# проверим, какие варианты длины кодов есть
df['zip_len'].value_counts()

5     372101
4       1893
10       238
6          3
8          1
9          1
Name: zip_len, dtype: int64

In [201]:
# судя по всему, индексы с длиной 4 знака вносили как числовой формат
# Для исправления ситуации добавим в начале укороченного индекса "0" и проверим их наличие еще раз
df['zipcode'] = df.apply(lambda row: row['zipcode'] if row['zip_len']>=5 else ('0'+row['zipcode']), axis=1)
df['zip_len'] = df['zipcode'].apply(lambda x: len(x))
df['zip_len'].value_counts()

5     373994
10       238
6          3
8          1
9          1
Name: zip_len, dtype: int64

In [202]:
# посмотрим на немногочисленные варианты длины индекса от 6 до 9 знаков
df[(df['zip_len']>5)&(df['zip_len']<10)]

,status,propertyType,street,baths,homeFacts,fireplace,city,schools,sqft,zipcode,beds,state,target,PoolPrivate,MLS,zip_len
85581,Active,"Detached, Two Story",6789 White River Home,0.00,"{'atAGlanceFacts': [{'factValue': '1989', 'fac...",NaN,other city value - out of area,"[{'rating': [], 'data': {'Distance': [], 'Grad...","4,000",123456,NaN,NY,899999,0,a10764075,6
112513,Active,"Charleston, Transitional",1307 Formal Garden Way,0.00,"{'atAGlanceFacts': [{'factValue': '2014', 'fac...",1,raleigh,"[{'rating': ['3', '3', '3'], 'data': {'Distanc...","2,399",27603-43,NaN,NC,352000,0,2292160,8
133901,Active,Apartment,2315 Gerritsen Ave Unit 3A,0.00,"{'atAGlanceFacts': [{'factValue': '2001', 'fac...",NaN,brooklyn,"[{'rating': ['9', '4', '7', 'NR'], 'data': {'D...","1,100",112229,NaN,NY,599000,0,435007,6
161389,Active,High Rise,301 Altara #402,0.00,"{'atAGlanceFacts': [{'factValue': '2019', 'fac...",NaN,coral gables,"[{'rating': ['7', 'NR', '10', '5', '5'], 'data...","1,212",331446,NaN,FL,620000,0,a10699951,6
242855,Active,NaN,132 Edward Booth Ln,2.25,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",NaN,durham,"[{'rating': ['4', '4', '4'], 'data': {'Distanc...",NaN,27713-967,Baths,NC,295000,0,2266947,9


In [203]:
# есть два с дефисом, которые первой своей частью выглядят очень натурально, как индекс
# остальные три проще будет удалить
# но сначала посмотрим на структуру 10-значных индексов, их побольше
df[df['zip_len']==10].head()

,status,propertyType,street,baths,homeFacts,fireplace,city,schools,sqft,zipcode,beds,state,target,PoolPrivate,MLS,zip_len
3654,Active,Log Home,718 Powell Dr,1.50,"{'atAGlanceFacts': [{'factValue': '1928', 'fac...",1,raleigh,"[{'rating': ['4', '8', '6'], 'data': {'Distanc...","1,936",27606-1626,Baths,NC,315000,0,2269185,10
5192,Active,Bungalow,915 Lake Boone Trl,2.50,"{'atAGlanceFacts': [{'factValue': '1950', 'fac...",1,raleigh,"[{'rating': ['6', '7', '5'], 'data': {'Distanc...","1,742",27607-6632,Baths,NC,475000,0,2241307,10
7155,Active,"Bungalow, Ranch",1109 Virginia Ave,1.75,"{'atAGlanceFacts': [{'factValue': '1940', 'fac...",1,durham,"[{'rating': ['4', '3', '4'], 'data': {'Distanc...","1,194",27705-3261,Baths,NC,284900,0,2281085,10
7545,Active,Traditional,3117 Henslowe Dr,0.00,"{'atAGlanceFacts': [{'factValue': '1986', 'fac...",1,raleigh,"[{'rating': ['5', '4', '6'], 'data': {'Distanc...","1,674",27603-2692,NaN,NC,245000,0,2292632,10
8586,Active,Transitional,1209 Graedon Dr,0.00,"{'atAGlanceFacts': [{'factValue': '2014', 'fac...",3,raleigh,"[{'rating': ['5', '4', '6'], 'data': {'Distanc...","6,517",27603-3986,NaN,NC,1599000,0,2252547,10


In [204]:
# уберем дефисы и последующие за ним знаки
df['zipcode'] = df['zipcode'].str.replace("-.+",'', regex=True)
df['zip_len'] = df['zipcode'].apply(lambda x: len(x))
df['zip_len'].value_counts()

5    374234
6         3
Name: zip_len, dtype: int64

In [205]:
# удалим 3 записи с 6-значными индексами
df = df[df['zip_len']==5]
df.shape[0]

374234

In [206]:
# подгрузим информацию из доп.файла о средневзвешенных координатах территории, закрепленной за почтовым индексом
zip_db = pd.read_csv('data/zip_code_database.csv', dtype='str')
zip_db.head()

,zip,type,primary_city,acceptable_cities,unacceptable_cities,state,county,timezone,area_codes,latitude,longitude,world_region,country,decommissioned,estimated_population,notes
0,00501,UNIQUE,Holtsville,NaN,I R S Service Center,NY,Suffolk County,America/New_York,631,40.81,-73.04,NaN,US,0,384,NaN
1,00544,UNIQUE,Holtsville,NaN,Irs Service Center,NY,Suffolk County,America/New_York,631,40.81,-73.04,NaN,US,0,0,NaN
2,00601,STANDARD,Adjuntas,NaN,"Colinas Del Gigante, Jard De Adjuntas, Urb San...",PR,Adjuntas,America/Puerto_Rico,"787,939",18.16,-66.72,NaN,US,0,0,NaN
3,00602,STANDARD,Aguada,NaN,"Alts De Aguada, Bo Guaniquilla, Comunidad Las ...",PR,NaN,NaN,787,18.38,-67.18,NaN,US,0,0,NaN
4,00603,STANDARD,Aguadilla,Ramey,"Bda Caban, Bda Esteves, Bo Borinquen, Bo Ceiba...",PR,Aguadilla,America/Puerto_Rico,787,18.43,-67.15,NaN,US,0,0,NaN


In [207]:
# левым соединением добавим к основному датасету координаты, оценим количество записей без координат
df = df.merge(zip_db[['zip','latitude','longitude']], 'left', left_on='zipcode', right_on='zip')
df['zip'].isna().sum()

141

In [208]:
# считаю допустимым в рамках этой работы удалить записи, к которым не сопоставились координаты
df = df[~df['zip'].isna()]
df.shape[0]

374093

In [209]:
# удалим теперь уже ненужные столбцы zip_len и zip, а также поменяем тип данных в столбцах с координатами
df = df.drop(['zip_len','zip'],axis=1)
df[['latitude', 'longitude']] = df[['latitude', 'longitude']].astype(float)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 374093 entries, 0 to 374233
Data columns (total 17 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   status        334841 non-null  object 
 1   propertyType  339548 non-null  object 
 2   street        372265 non-null  object 
 3   baths         374093 non-null  float64
 4   homeFacts     374093 non-null  object 
 5   fireplace     102333 non-null  object 
 6   city          374059 non-null  object 
 7   schools       374093 non-null  object 
 8   sqft          334031 non-null  object 
 9   zipcode       374093 non-null  object 
 10  beds          283228 non-null  object 
 11  state         374093 non-null  object 
 12  target        374093 non-null  int64  
 13  PoolPrivate   374093 non-null  int64  
 14  MLS           374017 non-null  object 
 15  latitude      374093 non-null  float64
 16  longitude     374093 non-null  float64
dtypes: float64(3), int64(2), object(12)
memory usage

__________

##### fireplace - всё о каминах

In [210]:
# для начала переведем все записи в нижний регистр
df['fireplace'] = df['fireplace'].str.lower()
# оценим пропуски и разнообразие вариантов
print('пропусков : ',df['fireplace'].isna().sum())
print('пропусков в %: ',round((df['fireplace'].isna().sum()/df.shape[0]*100),0),'%')
print('уникальных значений : ',df['fireplace'].nunique())

пропусков :  271760
пропусков в %:  73.0 %
уникальных значений :  1650


In [211]:
# похоже, в этом поле общераспространенные типовые варианты отсутствуют, каждый - свободный художник (и это ужасно)
# посмотрим наиболее ходовые формулировки
df['fireplace'].value_counts().head(100).tail(50)

walk-in closets                                                    49
1 fireplace, fireplace living rm                                   48
ceiling fan, utility connection, walk-in closets                   47
gas log, 1 fireplace                                               47
n/k                                                                47
1 fireplace, gas fireplace                                         45
fireplace-gas, forced air                                          45
ceiling fan, extra closets, utility connection, walk-in closets    44
familyrm, wood burning                                             44
ceiling fan, storage, utility connection, walk-in closets          40
ceiling fan, utility connection                                    39
1 fireplace, fireplace family rm, gas fireplace                    39
ceiling fan, storage, walk-in closets                              37
familyrm, gas vented                                               36
6                   

In [212]:
# всё намного хуже, поле зачастую использовалось для внесения дополнительной информации в свободной форме
# к сожалению, предполагаемые трудозатраты по обработке несопоставимы с эффектом от этого признака
# есть описания наличия шкафов, доп.оборудования в квартире, никак не связанных с каминами
# удаляем признак
df = df.drop(['fireplace'], axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 374093 entries, 0 to 374233
Data columns (total 16 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   status        334841 non-null  object 
 1   propertyType  339548 non-null  object 
 2   street        372265 non-null  object 
 3   baths         374093 non-null  float64
 4   homeFacts     374093 non-null  object 
 5   city          374059 non-null  object 
 6   schools       374093 non-null  object 
 7   sqft          334031 non-null  object 
 8   zipcode       374093 non-null  object 
 9   beds          283228 non-null  object 
 10  state         374093 non-null  object 
 11  target        374093 non-null  int64  
 12  PoolPrivate   374093 non-null  int64  
 13  MLS           374017 non-null  object 
 14  latitude      374093 non-null  float64
 15  longitude     374093 non-null  float64
dtypes: float64(3), int64(2), object(11)
memory usage: 48.5+ MB


____________

##### Beds - спальни

In [213]:
# для начала переведем все записи в нижний регистр
df['beds'] = df['beds'].str.lower()
# оцениваем количество пропусков и смотрим разнообразие вариантов значений признака
print('пропусков : ',df['beds'].isna().sum())
print('пропусков в %: ',round((df['beds'].isna().sum()/df.shape[0]*100),0),'%')
print('уникальных значений : ',df['beds'].nunique())

пропусков :  90865
пропусков в %:  24.0 %
уникальных значений :  1144


In [214]:
# посмотрим примеры значений
df['beds'].value_counts()

3 beds        52926
4 beds        35105
3             31187
2 beds        26074
4             19913
              ...  
3.8 acres         1
7,104 sqft        1
9,443 sqft        1
5.11 acres        1
8,479 sqft        1
Name: beds, Length: 1144, dtype: int64

In [215]:
# неожиданно нашлись значения площади с обозначениями acres и sqft
# оценим количество таких данных в поле beds и одновременное отсутствие в поле sqft
print(df[(df['beds'].str.contains('sqft', na=False)) | (df['beds'].str.contains('acres', na=False))].shape[0])
df[(df['beds'].str.contains('sqft', na=False)) | (df['beds'].str.contains('acres', na=False))]['sqft'].isna().sum()

2925


2894

In [216]:
# странным будет не вернуть эти данные туда, где им самое место - в поле sqft
def beds_sqft(beds, sqft):
    if type(beds) is float:
        return sqft
    if 'sqft' in beds:
        return beds
#    if 'acres' in beds: - не относится к sqft
#        return beds     - не относится к sqft
    return sqft

df['sqft'] = df.apply(lambda row: beds_sqft(row['beds'], row['sqft']), axis=1)

In [217]:
# теперь очистим в поле beds записи с площадью и проверим успешность этого мероприятия
df['beds'] = df['beds'].apply(lambda x: x if type(x) is float else np.nan if ('sqft' in x) or ('acres' in x) else x)
print(df[(df['beds'].str.contains('sqft', na=False)) | (df['beds'].str.contains('acres', na=False))].shape[0])

0


In [218]:
# уберем все пробелы и буквы, а также оставленные кем-то сочинения в этом поле
df['beds_len'] = df['beds'].apply(lambda x: 1.0 if type(x) is float else len(x))
df['beds'] = df.apply(lambda x: '0' if x['beds_len']>= 10.0 else x['beds'], axis=1)
df['beds'] = df['beds'].str.replace('[a-zA-Z]','', regex=True)
df['beds'] = df['beds'].apply(lambda x: 0 if x=='' else x)
df['beds'] = df['beds'].str.replace('--','0')
df['beds'] = df['beds'].str.replace(' ','')
df['beds'] = df['beds'].str.replace('#','')
df = df.drop('beds_len', axis=1)

In [219]:
# заполним остальные пробелы нулями
df['beds'] = df['beds'].fillna('0')
df['beds'] = df['beds'].astype(float)

___________

##### sqft - площадь в футах

In [220]:
# для начала переведем все записи в нижний регистр
df['sqft'] = df['sqft'].str.lower()
# оцениваем количество пропусков и смотрим разнообразие вариантов значений признака
print('пропусков : ',df['sqft'].isna().sum())
print('пропусков в %: ',round((df['sqft'].isna().sum()/df.shape[0]*100),0),'%')
print('уникальных значений : ',df['sqft'].nunique())
df['sqft'].value_counts()

пропусков :  38712
пропусков в %:  10.0 %
уникальных значений :  25550


0              11852
1,200 sqft       824
1,000 sqft       643
1,100 sqft       566
1,800 sqft       558
               ...  
5,177              1
11620              1
46,118 sqft        1
6,523 sqft         1
8,479 sqft         1
Name: sqft, Length: 25550, dtype: int64

In [221]:
# уберем часто встречающиеся символы и фразы, а также пока переведем в явные пробелы нулевые значения
# list_sqft = ['total interior livable area: ', ' sqft', ',']
df['sqft'] = df['sqft'].str.replace(' sqft', '').\
                            str.replace('total interior livable area: ', '').\
                            str.replace(',', '').\
                            str.replace('-', '').replace('0',np.nan).replace('',np.nan)
df['sqft'].value_counts()

1200      1406
1000      1013
1500      1010
1800       979
1100       929
          ... 
44840        1
288588       1
7728         1
6454         1
8479         1
Name: sqft, Length: 10024, dtype: int64

In [222]:
df['sqft'] = df['sqft'].astype(float)

______________

##### homeFacts - сведения о строительстве объекта

In [223]:
# представляет из себя составной признак, который имеет четкую структуру
# посмотрим внимательнее и разберем ее на составляющие
df['homeFacts'][0]

"{'atAGlanceFacts': [{'factValue': '2019', 'factLabel': 'Year built'}, {'factValue': '', 'factLabel': 'Remodeled year'}, {'factValue': 'Central A/C, Heat Pump', 'factLabel': 'Heating'}, {'factValue': '', 'factLabel': 'Cooling'}, {'factValue': '', 'factLabel': 'Parking'}, {'factValue': None, 'factLabel': 'lotsize'}, {'factValue': '$144', 'factLabel': 'Price/sqft'}]}"

In [224]:
# сформируем список с наименованиями вложенных признаков
new_columns = df['homeFacts'].str.findall(r"\bfactLabel': '([\s\S]+?)[}']")
new_columns[0]

['Year built',
 'Remodeled year',
 'Heating',
 'Cooling',
 'Parking',
 'lotsize',
 'Price/sqft']

In [225]:
# сформируем массив со значениями признаков с сохранением структуры
new_values = df['homeFacts'].str.findall(r"\bfactValue': ([\s\S]+?), 'factLabel\b")
new_values.head(5)

0    ['2019', '', 'Central A/C, Heat Pump', '', '',...
1    ['2019', '', '', '', '', '5828 sqft', '$159/sq...
2    ['1961', '1967', 'Forced Air', 'Central', 'Att...
3    ['2006', '2006', 'Forced Air', 'Central', 'Det...
4            ['', '', '', '', '', '10,019 sqft', None]
Name: homeFacts, dtype: object

In [226]:
# добавим извлеченные признаки в основной набор данных
for index, feature in enumerate(new_columns[0]):
    df[feature]=new_values.apply(lambda x: x[index].replace("'",""))
    
df.head()

,status,propertyType,street,baths,homeFacts,city,schools,sqft,zipcode,beds,...,MLS,latitude,longitude,Year built,Remodeled year,Heating,Cooling,Parking,lotsize,Price/sqft
0,Active,Single Family Home,240 Heather Ln,3.5,"{'atAGlanceFacts': [{'factValue': '2019', 'fac...",southern pines,"[{'rating': ['4', '4', '7', 'NR', '4', '7', 'N...",2900.0,28387,4.0,...,611019,35.18,-79.40,2019,,"Central A/C, Heat Pump",,,None,$144
1,for sale,single-family home,12911 E Heroy Ave,3.0,"{'atAGlanceFacts': [{'factValue': '2019', 'fac...",spokane valley,"[{'rating': ['4/10', 'None/10', '4/10'], 'data...",1947.0,99216,3.0,...,201916904,47.69,-117.19,2019,,,,,5828 sqft,$159/sqft
2,for sale,single-family home,2005 Westridge Rd,2.0,"{'atAGlanceFacts': [{'factValue': '1961', 'fac...",los angeles,"[{'rating': ['8/10', '4/10', '8/10'], 'data': ...",3000.0,90049,3.0,...,fr19221027,34.08,-118.49,1961,1967,Forced Air,Central,Attached Garage,"8,626 sqft",$965/sqft
3,for sale,single-family home,4311 Livingston Ave,8.0,"{'atAGlanceFacts': [{'factValue': '2006', 'fac...",dallas,"[{'rating': ['9/10', '9/10', '10/10', '9/10'],...",6457.0,75205,5.0,...,14191809,32.79,-96.76,2006,2006,Forced Air,Central,Detached Garage,"8,220 sqft",$371/sqft
4,for sale,lot/land,1524 Kiscoe St,0.0,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",palm bay,"[{'rating': ['4/10', '5/10', '5/10'], 'data': ...",NaN,32908,0.0,...,861745,27.98,-80.66,,,,,,"10,019 sqft",None


In [227]:
# поле homeFacts удаляем, т.к. вся необходимая информация из него извлечена
df = df.drop('homeFacts', axis=1)
# сразу можно удалить во избежание утечки признак Price/sqft
df = df.drop('Price/sqft', axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 374093 entries, 0 to 374233
Data columns (total 21 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   status          334841 non-null  object 
 1   propertyType    339548 non-null  object 
 2   street          372265 non-null  object 
 3   baths           374093 non-null  float64
 4   city            374059 non-null  object 
 5   schools         374093 non-null  object 
 6   sqft            322641 non-null  float64
 7   zipcode         374093 non-null  object 
 8   beds            374093 non-null  float64
 9   state           374093 non-null  object 
 10  target          374093 non-null  int64  
 11  PoolPrivate     374093 non-null  int64  
 12  MLS             374017 non-null  object 
 13  latitude        374093 non-null  float64
 14  longitude       374093 non-null  float64
 15  Year built      374093 non-null  object 
 16  Remodeled year  374093 non-null  object 
 17  Heating   

___________

##### Year built - год постройки

In [228]:
# для начала переведем все записи в нижний регистр
df['Year built'] = df['Year built'].str.lower()
# оцениваем количество пропусков и смотрим разнообразие вариантов значений признака
print('пропусков : ',df['Year built'].isna().sum())
print('пропусков в %: ',round((df['Year built'].isna().sum()/df.shape[0]*100),0),'%')
print('уникальных значений : ',df['Year built'].nunique())
df['Year built'].value_counts()

пропусков :  0
пропусков в %:  0.0 %
уникальных значений :  229


        58462
2019    32118
2006     7934
2005     7439
2007     7079
        ...  
1809        1
1823        1
1807        1
1740        1
1804        1
Name: Year built, Length: 229, dtype: int64

In [229]:
df['Year built'].sort_values(ascending=False).head(5)

68074     none
233094    none
2424      none
93566     none
301310    none
Name: Year built, dtype: object

In [230]:
df['Year built'].sort_values(ascending=True).head(5)

182277    
160163    
201335    
172246    
201330    
Name: Year built, dtype: object

In [231]:
# пока переведем нереальный год постройки и пропуски в явный вид отсутствия данных
df['Year built'] = df['Year built'].apply(lambda x: np.nan if 
                                          (x=='' or x=='1' or x=='none' or x=='no data' or x=='559990649990') 
                                          else x)

__________

##### Remodeled year - год реконструкции

In [232]:
# для начала переведем все записи в нижний регистр
df['Remodeled year'] = df['Remodeled year'].str.lower()
# оцениваем количество пропусков и смотрим разнообразие вариантов значений признака
print('пропусков : ',df['Remodeled year'].isna().sum())
print('пропусков в %: ',round((df['Remodeled year'].isna().sum()/df.shape[0]*100),0),'%')
print('уникальных значений : ',df['Remodeled year'].nunique())
df['Remodeled year'].value_counts()

пропусков :  0
пропусков в %:  0.0 %
уникальных значений :  155


        198008
none     26331
2006      5485
2005      4765
2007      4355
         ...  
1888         1
1800         1
1738         1
1869         1
1862         1
Name: Remodeled year, Length: 155, dtype: int64

In [233]:
# заменим непонятные значения на явные пропуски
df['Remodeled year'] = df['Remodeled year'].apply(lambda x: np.nan if 
                                          (x=='' or x=='0' or x=='none' or x=='1111') 
                                          else x)

In [234]:
df[~df['Remodeled year'].isna()]['Remodeled year'].sort_values()

304149    1738
293012    1800
153440    1845
145150    1846
250844    1853
          ... 
55387     2021
367550    2021
168496    2021
352974    2021
176113    2021
Name: Remodeled year, Length: 149484, dtype: object

_____________

##### Heating - информация об отоплении

In [235]:
# для начала переведем все записи в нижний регистр
df['Heating'] = df['Heating'].str.lower()
# оцениваем количество пропусков и смотрим разнообразие вариантов значений признака
print('пропусков : ',df['Heating'].isna().sum())
print('пропусков в %: ',round((df['Heating'].isna().sum()/df.shape[0]*100),0),'%')
print('уникальных значений : ',df['Heating'].nunique())
df['Heating'].value_counts()

пропусков :  0
пропусков в %:  0.0 %
уникальных значений :  1904


forced air                                                      133358
                                                                104873
other                                                            29050
electric                                                         10173
gas                                                               9261
                                                                 ...  
baseboard, spacewallunit                                             1
hot air, stove-pellet                                                1
natural gas, space heater                                            1
exhaust fans                                                         1
baseboard, hot water, programmable thermostat, radiant floor         1
Name: Heating, Length: 1904, dtype: int64

In [236]:
# заменим непонятные значения на явные пропуски
df['Heating'] = df['Heating'].apply(lambda x: np.nan if 
                                          (x=='' or x=='other' or x=='no data' or x=='none') 
                                          else x)

In [237]:
df[df['Heating'].isna()]['propertyType'].value_counts()

single-family home                                             28111
Single Family                                                  17643
lot/land                                                       14639
Land                                                           10924
Single Family Home                                             10415
                                                               ...  
Single Detached, Contemporary/Modern, Ranch, Traditional           1
30X40 building                                                     1
Bungalow, Craftsman, Ranch, Transitional, Cottage                  1
Condo/Townhome, Contemporary/Modern, Split Level                   1
Bilevel, Converted Dwelling, Loft with Bedrooms, Condo/Unit        1
Name: propertyType, Length: 721, dtype: int64

___________

##### Cooling - системы охлаждения

In [238]:
# для начала переведем все записи в нижний регистр
df['Cooling'] = df['Cooling'].str.lower()
# оцениваем количество пропусков и смотрим разнообразие вариантов значений признака
print('пропусков : ',df['Cooling'].isna().sum())
print('пропусков в %: ',round((df['Cooling'].isna().sum()/df.shape[0]*100),0),'%')
print('уникальных значений : ',df['Cooling'].nunique())
df['Cooling'].value_counts()

пропусков :  0
пропусков в %:  0.0 %
уникальных значений :  1437


central                                                                        157530
                                                                               119108
central air                                                                     14382
none                                                                            10774
no data                                                                         10395
                                                                                ...  
other (see remarks), panel/floor/wall, window unit                                  1
multi units, zoned cooling                                                          1
central air, g-energy star hvac, gas hot air/furnace, multizone a/c                 1
g-fresh air ventilation, gas hot air/furnace, heat pump - ac, multizone a/c         1
central a/c (gas), central heat (gas), heat pump                                    1
Name: Cooling, Length: 1437, dtype: int64

In [239]:
# заменим непонятные значения на явные пропуски
df['Cooling'] = df['Cooling'].apply(lambda x: np.nan if 
                                          (x=='' or x=='none' or x=='no data' or x=='other') 
                                          else x)

In [240]:
df['Cooling'].value_counts().head(50)

central                                                        157530
central air                                                     14382
has cooling                                                      9726
central electric                                                 6154
wall                                                             3981
central gas                                                      3573
central heating                                                  2807
cooling system                                                   2700
central a/c                                                      2051
central a/c (electric), central heat (gas)                       1646
central a/c (electric), central heat (electric)                  1429
refrigeration                                                    1061
central, electric                                                1059
electric                                                         1009
evaporative         

___________

##### Parking - информация о парковке

In [241]:
# для начала переведем все записи в нижний регистр
df['Parking'] = df['Parking'].str.lower()
# оцениваем количество пропусков и смотрим разнообразие вариантов значений признака
print('пропусков : ',df['Parking'].isna().sum())
print('пропусков в %: ',round((df['Parking'].isna().sum()/df.shape[0]*100),0),'%')
print('уникальных значений : ',df['Parking'].nunique())
df['Parking'].value_counts()

пропусков :  0
пропусков в %:  0.0 %
уникальных значений :  3318


                                                                                                                                    170756
attached garage                                                                                                                      70182
2 spaces                                                                                                                             27831
1 space                                                                                                                              14099
detached garage                                                                                                                      13088
                                                                                                                                     ...  
attached - rear, aspht, unassigned                                                                                                       1
bath in garage, boat, circu

In [242]:
# заменим непонятные значения на явные пропуски
df['Parking'] = df['Parking'].apply(lambda x: np.nan if 
                                          (x=='' or x=='none' or x=='no data' or x=='other') 
                                          else x)

__________

##### lotsize - площадь участка

In [243]:
# для начала переведем все записи в нижний регистр
df['lotsize'] = df['lotsize'].str.lower()
# оцениваем количество пропусков и смотрим разнообразие вариантов значений признака
print('пропусков : ',df['lotsize'].isna().sum())
print('пропусков в %: ',round((df['lotsize'].isna().sum()/df.shape[0]*100),0),'%')
print('уникальных значений : ',df['lotsize'].nunique())
df['lotsize'].value_counts()

пропусков :  0
пропусков в %:  0.0 %
уникальных значений :  36542


                  32447
none              28193
—                 25174
no data            5292
-- sqft lot        3809
                  ...  
2,921 sqft            1
3,686 sq. ft.         1
8,479 sq. ft.         1
5,573 sq. ft.         1
7,084 sqft lot        1
Name: lotsize, Length: 36542, dtype: int64

In [244]:
# заменим непонятные значения на нулевые значения
df['lotsize'] = df['lotsize'].apply(lambda x: 0 if 
                                          (x=='' or x=='none' or x=='no data' or x=='—' or x=='-- sqft lot') 
                                          else x)

In [245]:
# уберем часто встречающиеся символы и фразы
df['lotsize'] = df['lotsize'].str.replace(' sqft', '').\
                            str.replace(' lot', '').\
                            str.replace(' sq. ft.', '').\
                            str.replace('no data', '').\
                            str.replace('none', '').\
                            str.replace(',', '').\
                            str.replace('-', '').\
                            str.replace('-', '').replace('0',np.nan).replace('',np.nan)

/var/folders/0v/d10yh6tj1mn387jzy55lmxv80000gn/T/ipykernel_1061/3372141234.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['lotsize'] = df['lotsize'].str.replace(' sqft', '').\


In [246]:
# пересчитаем акры в квадратные футы через дополнительный столбец, и в конце удалим его
df['lot_acres'] = df['lotsize'].apply(lambda x: np.nan if type(x)==float else x if 'acr' in x else np.nan)
df['lot_acres'] = df['lot_acres'].str.replace(' acres','').\
                                str.replace(' acre','')
df['lot_acres'] = df['lot_acres'].astype(float)
df['lotsize'] = df.apply(lambda row: row['lot_acres']*43560 if row['lot_acres']>0 else row['lotsize'], axis=1)
df['lotsize'] = df['lotsize'].astype(float)
df = df.drop('lot_acres', axis=1)

________

##### schools - информация о ближайших школах

In [247]:
# список свернутых воедино признаков
df['schools'][5]

"[{'rating': [], 'data': {'Distance': [], 'Grades': []}, 'name': []}]"

In [248]:
# рассчитаем минимальное расстояние для школы и средний рейтинг школ в ближайшем окружении
# наименование школы при массовой обработке смысла не несет
# начнем с рейтинга
school_rate = df['schools'].str.findall(r"\brating': ([\s\S]+?), 'data\b")
school_rate = school_rate.apply(lambda x: x[0])
school_rate

0         ['4', '4', '7', 'NR', '4', '7', 'NR', 'NR']
1                         ['4/10', 'None/10', '4/10']
2                            ['8/10', '4/10', '8/10']
3                   ['9/10', '9/10', '10/10', '9/10']
4                            ['4/10', '5/10', '5/10']
                             ...                     
374229                              ['10/10', '5/10']
374230                       ['1/10', '5/10', '7/10']
374231                               ['5/10', '4/10']
374232                             ['NA', 'NA', 'NA']
374233                       ['5/10', '4/10', '3/10']
Name: schools, Length: 374093, dtype: object

In [249]:
# уберем явные комбинации символов, все буквы, и вычислим среднее
school_rate = school_rate.str.replace('/10','')
school_rate = school_rate.str.replace('[a-zA-Z]','', regex=True)
school_rate = school_rate.str.findall(r'\b([0-9])\'')
school_rate = school_rate.apply(lambda x: [float(i) for i in x])
school_rate = school_rate.apply(lambda x: np.average([int(i) for i in x]))
df['school_rate'] = school_rate.round(1)
df.head()

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/numpy/lib/function_base.py:518: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)


,status,propertyType,street,baths,city,schools,sqft,zipcode,beds,state,...,MLS,latitude,longitude,Year built,Remodeled year,Heating,Cooling,Parking,lotsize,school_rate
0,Active,Single Family Home,240 Heather Ln,3.5,southern pines,"[{'rating': ['4', '4', '7', 'NR', '4', '7', 'N...",2900.0,28387,4.0,NC,...,611019,35.18,-79.40,2019,NaN,"central a/c, heat pump",NaN,NaN,NaN,5.2
1,for sale,single-family home,12911 E Heroy Ave,3.0,spokane valley,"[{'rating': ['4/10', 'None/10', '4/10'], 'data...",1947.0,99216,3.0,WA,...,201916904,47.69,-117.19,2019,NaN,NaN,NaN,NaN,5828.0,4.0
2,for sale,single-family home,2005 Westridge Rd,2.0,los angeles,"[{'rating': ['8/10', '4/10', '8/10'], 'data': ...",3000.0,90049,3.0,CA,...,fr19221027,34.08,-118.49,1961,1967,forced air,central,attached garage,8626.0,6.7
3,for sale,single-family home,4311 Livingston Ave,8.0,dallas,"[{'rating': ['9/10', '9/10', '10/10', '9/10'],...",6457.0,75205,5.0,TX,...,14191809,32.79,-96.76,2006,2006,forced air,central,detached garage,8220.0,9.0
4,for sale,lot/land,1524 Kiscoe St,0.0,palm bay,"[{'rating': ['4/10', '5/10', '5/10'], 'data': ...",NaN,32908,0.0,FL,...,861745,27.98,-80.66,NaN,NaN,NaN,NaN,NaN,10019.0,4.7


In [250]:
school_dist = df['schools'].str.findall(r"\bDistance': ([\s\S]+?), 'Grades\b")
school_dist = school_dist.apply(lambda x: x[0])
school_dist = school_dist.str.replace('[a-zA-Z]','', regex=True)
school_dist = school_dist.str.findall(r'\b([0-9]+.[0-9]+)')
school_dist = school_dist.apply(lambda x: [float(i) for i in x])
school_dist = school_dist.apply(lambda x: -1 if len(x)==0 else min(x))
df['school_dist'] = school_dist.round(1)
df.head()

,status,propertyType,street,baths,city,schools,sqft,zipcode,beds,state,...,latitude,longitude,Year built,Remodeled year,Heating,Cooling,Parking,lotsize,school_rate,school_dist
0,Active,Single Family Home,240 Heather Ln,3.5,southern pines,"[{'rating': ['4', '4', '7', 'NR', '4', '7', 'N...",2900.0,28387,4.0,NC,...,35.18,-79.40,2019,NaN,"central a/c, heat pump",NaN,NaN,NaN,5.2,2.7
1,for sale,single-family home,12911 E Heroy Ave,3.0,spokane valley,"[{'rating': ['4/10', 'None/10', '4/10'], 'data...",1947.0,99216,3.0,WA,...,47.69,-117.19,2019,NaN,NaN,NaN,NaN,5828.0,4.0,1.0
2,for sale,single-family home,2005 Westridge Rd,2.0,los angeles,"[{'rating': ['8/10', '4/10', '8/10'], 'data': ...",3000.0,90049,3.0,CA,...,34.08,-118.49,1961,1967,forced air,central,attached garage,8626.0,6.7,1.2
3,for sale,single-family home,4311 Livingston Ave,8.0,dallas,"[{'rating': ['9/10', '9/10', '10/10', '9/10'],...",6457.0,75205,5.0,TX,...,32.79,-96.76,2006,2006,forced air,central,detached garage,8220.0,9.0,0.1
4,for sale,lot/land,1524 Kiscoe St,0.0,palm bay,"[{'rating': ['4/10', '5/10', '5/10'], 'data': ...",NaN,32908,0.0,FL,...,27.98,-80.66,NaN,NaN,NaN,NaN,NaN,10019.0,4.7,3.0


In [251]:
# удаляем исходный признак
df = df.drop('schools', axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 374093 entries, 0 to 374233
Data columns (total 22 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   status          334841 non-null  object 
 1   propertyType    339548 non-null  object 
 2   street          372265 non-null  object 
 3   baths           374093 non-null  float64
 4   city            374059 non-null  object 
 5   sqft            322641 non-null  float64
 6   zipcode         374093 non-null  object 
 7   beds            374093 non-null  float64
 8   state           374093 non-null  object 
 9   target          374093 non-null  int64  
 10  PoolPrivate     374093 non-null  int64  
 11  MLS             374017 non-null  object 
 12  latitude        374093 non-null  float64
 13  longitude       374093 non-null  float64
 14  Year built      310890 non-null  object 
 15  Remodeled year  149484 non-null  object 
 16  Heating         228204 non-null  object 
 17  Cooling   

_____________

##### Status - статус продажи

In [252]:
# для начала переведем все записи в нижний регистр
df['status'] = df['status'].str.lower()
# оцениваем количество пропусков и смотрим разнообразие вариантов значений признака
print('пропусков : ',df['status'].isna().sum())
print('пропусков в %: ',round((df['status'].isna().sum()/df.shape[0]*100),0),'%')
print('уникальных значений : ',df['status'].nunique())
df['status'].value_counts()

пропусков :  39252
пропусков в %:  10.0 %
уникальных значений :  146


for sale                  199402
active                    105192
foreclosure                 6018
new construction            5448
pending                     4802
                           ...  
coming soon: dec 15.           1
coming soon: dec 25.           1
pending backups wanted         1
coming soon: nov 23.           1
coming soon: dec 23.           1
Name: status, Length: 146, dtype: int64

In [253]:
# произведем объединение в более обобщенные группы статусов
df['status'] = df['status'].apply(lambda x: x if (type(x) is float) else 'coming soon' if 'soon' in x else x)
df['status'] = df['status'].apply(lambda x: x if (type(x) is float) else 'foreclosure' if 'forecl' in x else x)
df['status'] = df['status'].apply(lambda x: x if (type(x) is float) else 'auction' if 'auct' in x else x)
df['status'] = df['status'].apply(lambda x: x if (type(x) is float) else 'new' if 'new' in x else x)
df['status'] = df['status'].apply(lambda x: x if (type(x) is float) else 'pending' if 'pendi' in x else x)
df['status'] = df['status'].apply(lambda x: x if (type(x) is float) else 'contingent' if 'conting' in x else x)
df['status'] = df['status'].apply(lambda x: x if (type(x) is float) else 'contract' if 'contr' in x else x)

list_other = ['p', 'a active', 'price change', 'back on market', 'active option', 'pf', 'c', 'pi',
       'due diligence period', 'c continue show', 'active with offer', 'listing extended', 'active backup', 
       'ct', 'temporary active', 'closed', 'accepted offer', 'accepting backups', 'lease/purchase',
       'ps', 're activated', 'reactivated', 'uc continue to show', 'backup']
df['status'] = df['status'].apply(lambda x: x if (type(x) is float) else 'other' if x in list_other else x)

df['status'].value_counts()

for sale       199402
active         105192
foreclosure      9595
new              6137
pending          5359
contract         3767
other            2879
auction          1330
contingent       1070
coming soon       110
Name: status, dtype: int64

_________________

##### propertyType - тип объекта недвижимости

In [254]:
# для начала переведем все записи в нижний регистр
df['propertyType'] = df['propertyType'].str.lower()
# оцениваем количество пропусков и смотрим разнообразие вариантов значений признака
print('пропусков : ',df['propertyType'].isna().sum())
print('пропусков в %: ',round((df['propertyType'].isna().sum()/df.shape[0]*100),0),'%')
print('уникальных значений : ',df['propertyType'].nunique())
df['propertyType'].value_counts()

пропусков :  34545
пропусков в %:  9.0 %
уникальных значений :  1271


single-family home                                             91093
single family                                                  61838
condo                                                          42353
single family home                                             31725
lot/land                                                       20503
                                                               ...  
custom, elevated, other                                            1
1 story, contemporary, traditional, mediterranean                  1
2 stories, traditional, mediterranean, texas hill country          1
loft, warehouse                                                    1
bilevel, converted dwelling, loft with bedrooms, condo/unit        1
Name: propertyType, Length: 1271, dtype: int64

In [255]:
# судя по структуре, будем считать, что основной смысл этого поля содержится до первой запятой
# выделим эту важную часть в отдельный столбец
df['PrType'] = df['propertyType'].str.split(',').str[0]
df['PrType'].unique()

array(['single family home', 'single-family home', 'lot/land',
       'townhouse', 'florida', nan, 'single family', 'coop', 'english',
       '2 story', 'multi-family', 'penthouse', 'multi-family home',
       'condo', 'land', 'condo/townhome/row home/co-op', '', 'detached',
       '1 story', 'other style', 'colonial', 'transitional', 'high rise',
       'mobile/manufactured', 'tri-level', 'craftsman', 'single detached',
       'federal', 'multi family', 'traditional', 'custom', 'cooperative',
       'contemporary/modern', 'cape cod', 'mobile / manufactured',
       'miscellaneous', 'mfd/mobile home', 'bungalow',
       'spanish/mediterranean', 'contemporary', 'multi-level',
       'condo/unit', '2 stories', 'ranch', 'low-rise (1-3 stories)',
       'rancher', 'urban contemporary', 'two story', 'garden home',
       'farms/ranches', 'a-frame', 'attached or 1/2 duplex', 'one story',
       'georgian', 'dwelling with rental', 'victorian', 'apartment',
       'cluster home', 'manufactured

In [256]:
# разделим это разнообразие признаков на группы, которые мне показались логичными по описаниям/названиям
# допускаю, что при большем опыте в этой сфере какие-то нюансы были бы более очевидны
single_family = ['single-family home', 'single family', 'single family home', 'singlefamilyresidence',
                 'single detached', 'single wide', 'single-wide mobile with land', 
                 'one story', 'two story', '1 story', '2 story', '2 stories', '1 story/ranch', '1 story traditional', 
                 '1 1/2 story', 'one story traditional', 'detached', 'single detached', 
                 'one level unit', 'rancher', 'single wide mh']
townhouse = ['townhouse', 'townhouse-interior', 'townhouse-end unit', 'townhome style']
modern = ['modern', 'modern farmhouse', 'modernist', 'contemporary', 'contemporary/modern', 'mid-century modern']
multi_family = ['multi-family', 'multi family', 'multi-family home', 'duplex', 'triplex', 'fourplex']
mobile = ['mobile/manufactured', 'mobile home 1 story', 'mobile manu - double wide', 
        'mfd/mobile home', 'mobile home', 'manufactured home', 'mobile / manufactured', 'manufactured house', 
         'manufactured double-wide', 'manufactured single-wide']
condo = ['condo', 'high rise', 'condo/townhome/row home/co-op', 'condo/townhome',
         'condominium (single level)', 'condo/unit', 'condominium',
         'condominium (2 levels)', 'condominium (3+ levels)',
         '2 unit condo', '2 story condo', 'coop', 'co-op', 'cooperative',
          'mid-rise', 'low-rise (1-3 stories)',]
land = ['land', 'lot/land']
apart = ['apartment',  'Flats', 'studio']
historic = ['historic', 'historical', 'designated historical home', 'historic vintage', 
            'historic/older', 'historical/conservation district', 'colonial']
ranch = ['ranch', 'garden home', 'farms/ranches']
prop_types = single_family + townhouse + modern + multi_family + mobile + condo + land + apart + historic + ranch


In [257]:
df['PrType'] = df['PrType'].apply(lambda x: x if x in prop_types else 'other')
df['PrType'] = df['PrType'].apply(lambda x: 'single family' if x in single_family else x)
df['PrType'] = df['PrType'].apply(lambda x: 'condominium' if x in condo else x)
df['PrType'] = df['PrType'].apply(lambda x: 'land' if x in land else x)
df['PrType'] = df['PrType'].apply(lambda x: 'townhouse' if x in townhouse else x)
df['PrType'] = df['PrType'].apply(lambda x: 'multi family' if x in multi_family else x)
df['PrType'] = df['PrType'].apply(lambda x: 'mobile' if x in mobile else x)
df['PrType'] = df['PrType'].apply(lambda x: 'modern' if x in modern else x)
df['PrType'] = df['PrType'].apply(lambda x: 'ranch' if x in ranch else x)
df['PrType'] = df['PrType'].apply(lambda x: 'apart' if x in apart else x)
df['PrType'] = df['PrType'].apply(lambda x: 'historic' if x in historic else x)
df['PrType'].value_counts()

single family    195244
condominium       57179
other             47922
land              31435
townhouse         18436
multi family      12163
mobile             3535
modern             3251
ranch              2743
historic           1390
apart               795
Name: PrType, dtype: int64

In [258]:
df = df.drop(['propertyType'], axis=1)

____________

##### Повторно пройдем по признакам с пропусками с учетом всех доработок

In [259]:
# отсортируем признаки по убыванию доли пустых значений
print('Всего записей в датасете : ',df.shape[0])
df.isna().sum().sort_values(ascending=False)

Всего записей в датасете :  374093


Remodeled year    224609
Parking           189602
Heating           145889
Cooling           142063
lotsize            94915
Year built         63203
sqft               51452
status             39252
school_rate         5745
street              1828
MLS                   76
city                  34
school_dist            0
latitude               0
longitude              0
PoolPrivate            0
target                 0
state                  0
beds                   0
zipcode                0
baths                  0
PrType                 0
dtype: int64

In [260]:
# Посмотрим срез по годам постройки и реставрации в предположении, что данные отсутствуют только у продаваемых участков
# Отсутствие года ремонта может обозначать его отсутствие для новых домов, например, но это можно выяснить 
# при наличии и года постройки, а с ними тоже проблема
df[(df['Remodeled year'].isna())&(df['Year built']).isna()]['PrType'].value_counts()

land             26934
other            16311
single family    14325
townhouse         2154
condominium       1556
multi family       190
ranch              154
apart              132
mobile              58
modern              30
historic            27
Name: PrType, dtype: int64

In [261]:
# неутешительно, на продажу участков земли приходится достаточно небольшая доля пропусков
# год ремонта - удаляем
df = df.drop('Remodeled year', axis=1)

In [262]:
# Паркинг проверим по типам объектов с тем же расчетом - могли не указывать к голым участкам
df[df['Parking'].isna()]['PrType'].value_counts()

single family    63539
other            47837
land             29585
condominium      25880
townhouse         7210
multi family      5794
modern            3251
ranch             2717
mobile            1768
historic          1390
apart              631
Name: PrType, dtype: int64

In [263]:
# Не верится, что в стране, где пешком почти никто не ходит, у половины домов в продаже отсутствует парковка
# Видимо, признак считают неважным и заполняют как попало. Удаляем.
df = df.drop('Parking', axis=1)

In [264]:
# Heating и Cooling - однотипная ситуация. Поля без выпадающих списков для выбора,
# куча рукописных вариантов, причем многое даже не относится к заполняемому параметру
# смысл этих полей вижу не в наличии/отсутствии отопления, т.к. странно представить 
# такого рода дома в массовой продаже, а скорее в определении типа отопления/охлаждения.
# К сожалению, объем трудозатрат оставляет эту проработку только в теории. Удаляем оба.
df = df.drop(['Heating','Cooling'], axis=1)

In [265]:
# Далее по рейтингу идут признаки с пробелами менее 30% от объема выборки, поэтому смотрим внимательнее.
# Год постройки заполнен далеко не во всех записях. Где-то обоснованно (продается участок), а где-то нет.
df[df['Year built'].isna()]['PrType'].value_counts()

land             26942
other            17569
single family    14342
townhouse         2157
condominium       1591
multi family       194
ranch              156
apart              133
mobile              58
modern              32
historic            29
Name: PrType, dtype: int64

In [266]:
# Достаточно большая часть пробелов - оправданно отсутствующие даты. Заполним значением 'no data'
df['Year built'] = df['Year built'].fillna('no data')

In [267]:
# Далее важный признак площади - sqft. Максимльно заполнили из других полей, где обнаруживались данные
df[df['sqft'].isna()]['PrType'].value_counts()

land             27333
other            17098
single family     3084
condominium       2012
multi family       874
townhouse          348
ranch              256
mobile             227
historic           155
apart               33
modern              32
Name: PrType, dtype: int64

In [268]:
# Остальное заполним нулями, т.к. в основном пробелы в площади дома касаются продаваемых участков
df['sqft'] = df['sqft'].fillna(0.0)

In [269]:
# В поле status заполним пробелы значением no status, т.к. дополнительной информации в этой части не добавилось
df['status'] = df['status'].fillna('no status')

In [270]:
df['street'].value_counts()

2103 E State Hwy 21      57
11305 Gulf Fwy           54
17030 Youngblood Rd.     38
NE 58th Cir              34
9470 Lancaster Rd. SW    32
                         ..
2705 S Beulah St          1
1398 Vander Ave           1
3439 Summerway Dr         1
1809 Bamboo St            1
7810 Pereida St           1
Name: street, Length: 298507, dtype: int64

In [271]:
df['street'].isna().sum()

1828

In [272]:
# пробелы в рейтингах школ заполним нулями
df['school_rate'] = df['school_rate'].fillna('0')
df['school_rate'] = df['school_rate'].astype(float)

In [273]:
# пробелы в адресе заполним значениями 'no address'
df['street'] = df['street'].fillna('no address')

In [274]:
# пробелы в городах заполним no city
df['city'] = df['city'].fillna('no city')

In [275]:
# пробелы в индексах MLS заполним no MLS
df['MLS'] = df['MLS'].fillna('no MLS')

In [276]:
# пробелы в lotsize заполним нулями
df['lotsize'] = df['lotsize'].fillna(0.0)

In [277]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 374093 entries, 0 to 374233
Data columns (total 18 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   status       374093 non-null  object 
 1   street       374093 non-null  object 
 2   baths        374093 non-null  float64
 3   city         374093 non-null  object 
 4   sqft         374093 non-null  float64
 5   zipcode      374093 non-null  object 
 6   beds         374093 non-null  float64
 7   state        374093 non-null  object 
 8   target       374093 non-null  int64  
 9   PoolPrivate  374093 non-null  int64  
 10  MLS          374093 non-null  object 
 11  latitude     374093 non-null  float64
 12  longitude    374093 non-null  float64
 13  Year built   374093 non-null  object 
 14  lotsize      374093 non-null  float64
 15  school_rate  374093 non-null  float64
 16  school_dist  374093 non-null  float64
 17  PrType       374093 non-null  object 
dtypes: float64(8), int64(2),

___________

In [278]:
# проверим, есть ли строки, в которых одновременно отсутствует информация о площади как дома, так и участка
df[(df['sqft'] == 0.0) & (df['lotsize'] == 0.0)].shape[0]

13678

In [279]:
# по этой комбинации нет никаких сомнений, эти записи бесполезны, удаляем
df = df[~((df['sqft'] == 0.0) & (df['lotsize'] == 0.0))]

In [280]:
# проверим датасет на появившиеся полные дубликаты
df.duplicated().sum()

1068

In [281]:
# удалим дубликаты
df = df.drop_duplicates()

In [282]:
# проверим наличие дублей при исключении из датасета того или иного поля
columns = df.columns
for column in columns:
    print(column, ' - ',df.drop(columns=column, axis=1).duplicated().sum())

status  -  1267
street  -  1063
baths  -  106
city  -  29
sqft  -  28
zipcode  -  0
beds  -  5
state  -  0
target  -  257
PoolPrivate  -  61
MLS  -  145
latitude  -  0
longitude  -  0
Year built  -  101
lotsize  -  1090
school_rate  -  374
school_dist  -  1293
PrType  -  28


In [283]:
# Удалим эти дубликаты, при этом оставим наиболее свежие записи из дублирующихся
# в предположении, что это максимально актуальная информация
df_status = list(df[df.drop('status', axis=1).duplicated(keep='last')].index)
df_street = list(df[df.drop('street', axis=1).duplicated(keep='last')].index)
df_baths = list(df[df.drop('baths', axis=1).duplicated(keep='last')].index)
df_city = list(df[df.drop('city', axis=1).duplicated(keep='last')].index)
df_sqft = list(df[df.drop('sqft', axis=1).duplicated(keep='last')].index)
df_beds = list(df[df.drop('beds', axis=1).duplicated(keep='last')].index)
df_target = list(df[df.drop('target', axis=1).duplicated(keep='last')].index)
df_PoolPrivate = list(df[df.drop('PoolPrivate', axis=1).duplicated(keep='last')].index)
df_MLS = list(df[df.drop('MLS', axis=1).duplicated(keep='last')].index)
df_Yearbuilt = list(df[df.drop('Year built', axis=1).duplicated(keep='last')].index)
df_lotsize = list(df[df.drop('lotsize', axis=1).duplicated(keep='last')].index)
df_school_rate = list(df[df.drop('school_rate', axis=1).duplicated(keep='last')].index)
df_school_dist = list(df[df.drop('school_dist', axis=1).duplicated(keep='last')].index)
df_PrType = list(df[df.drop('PrType', axis=1).duplicated(keep='last')].index)
index_set = set(df_status +
                df_street +
                df_baths +
                df_city +
                df_sqft +
                df_beds +
                df_target +
                df_PoolPrivate +
                df_MLS +
                df_Yearbuilt +
                df_lotsize +
                df_school_rate +
                df_school_dist +
                df_PrType)
len(index_set)

5842

In [284]:
# Удалим эти дубликаты, при этом оставим наиболее свежие записи из дублирующихся
# в предположении, что это максимально актуальная информация
df = df.drop(index_set, axis=0)

In [285]:
# проверим еще раз наличие дублей при исключении из датасета того или иного поля
columns = df.columns
for column in columns:
    print(column, ' - ',df.drop(columns=column, axis=1).duplicated().sum())

status  -  0
street  -  0
baths  -  0
city  -  0
sqft  -  0
zipcode  -  0
beds  -  0
state  -  0
target  -  0
PoolPrivate  -  0
MLS  -  0
latitude  -  0
longitude  -  0
Year built  -  0
lotsize  -  0
school_rate  -  0
school_dist  -  0
PrType  -  0


In [286]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 353505 entries, 0 to 374233
Data columns (total 18 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   status       353505 non-null  object 
 1   street       353505 non-null  object 
 2   baths        353505 non-null  float64
 3   city         353505 non-null  object 
 4   sqft         353505 non-null  float64
 5   zipcode      353505 non-null  object 
 6   beds         353505 non-null  float64
 7   state        353505 non-null  object 
 8   target       353505 non-null  int64  
 9   PoolPrivate  353505 non-null  int64  
 10  MLS          353505 non-null  object 
 11  latitude     353505 non-null  float64
 12  longitude    353505 non-null  float64
 13  Year built   353505 non-null  object 
 14  lotsize      353505 non-null  float64
 15  school_rate  353505 non-null  float64
 16  school_dist  353505 non-null  float64
 17  PrType       353505 non-null  object 
dtypes: float64(8), int64(2),

Будем считать первичную обработку датасета законченной.

In [287]:
df.to_csv('data/data_1.csv', index=False)

________________